In [3]:
import torch, dgl
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from dgl.data import DGLDataset
import re
import numpy as np
import pandas as pd
import os.path as osp
from glob import glob
from copy import deepcopy
from colorama import Fore
from random import shuffle
from typing import Tuple, Optional, Dict
import warnings
warnings.filterwarnings('ignore')

from graphSAGE import GraphSAGE
# from adabelief_pytorch import AdaBelief
from focal_loss import FocalLoss, focal_loss


labels = ['__ignore__', '_background_']

def create_cls_to_idx(label_path:str) -> Tuple[list, Dict[str, int]]:
    assert label_path.endswith('.txt')
    with open(label_path, 'r') as f:
        lines = f.read().strip().splitlines()
    classes = sorted([l for l in lines if l not in labels])
    
    with open(label_path, 'w') as f:
        f.write('\n'.join(labels + classes))
        
    class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
    return classes, class_to_idx

classes, class_to_idx = create_cls_to_idx('dataset/DKKD/labels.txt')

def class_2_idx(classname:str):
    assert classname in class_to_idx.keys(), 'invalid classname: ' + Fore.RED + classname
    return class_to_idx[classname]

##### 2. Graph Dataset

In [4]:
class DkkdGraphDataset(DGLDataset):
    def __init__(self, root:str='/home/agent/Documents/graph/GNN/dataset/DKKD_graph'):
        super().__init__(name='dataset/DKKD_graph')
        self.root = root
        self.edges = glob(osp.join(root, '*.edges.csv'))
        # self.nodes_feat = glob(osp.join(root, '*.nfeat.npy'))
        # self.nodes_label = glob(osp.join(root, '*.idx.csv'))
        self.shuffle()
    
    @staticmethod
    def _get_n_nodes(nodes_label:pd.DataFrame) -> int:
        r"""
        tính và kiểm tra số thứ tự của node
        """
        n_nodes = nodes_label['Id'].to_list()
        for i, idx in enumerate(n_nodes):
            assert i == idx, 'i != idx'
        return len(n_nodes)
    
    def __len__(self): return len(self.edges)
    
    def __getitem__(self, i) -> dgl.DGLGraph:
        edgep = self.edges[i]
        nodes_feat = np.load(re.sub('.edges.csv$', '.nfeat.npy', edgep))
        nodes_label = pd.read_csv(
            re.sub('.edges.csv$', '.idx.csv', edgep), encoding='utf-8')
        n_nodes = self._get_n_nodes(nodes_label)
        
        nodes_label = nodes_label['label'].astype('category').cat.codes.to_list()
        edge = pd.read_csv(edgep, encoding='utf-8')
        
        g = dgl.graph((edge['src'], edge['dst']), num_nodes=n_nodes)
        g = dgl.to_bidirected(g)
        g = dgl.remove_self_loop(g)
        g = dgl.add_self_loop(g)
        g.ndata['feat' ] = torch.from_numpy(nodes_feat )
        g.ndata['label'] = torch.tensor    (nodes_label)
        # g.ndata['train_mask'] = torch.ones (n_nodes, dtype=torch.bool)
        # g.ndata['val_mask'  ] = torch.zeros(n_nodes, dtype=torch.bool)
        # g.ndata['test_mask' ] = torch.zeros(n_nodes, dtype=torch.bool)
        
        return g
    
    def shuffle(self): shuffle(self.edges)
        
    def process(self): ...
    
train_data = DkkdGraphDataset(root='dataset/DKKD_graph')
val_data = DkkdGraphDataset(root='dataset/DKKD_graph_test')


##### 3. Training function

In [5]:
def cacu_alpha() -> torch.Tensor:
    df = pd.read_csv('dataset/DKKD/classes_unbalance.csv', encoding='utf-8')
    classes = df['class_name']
    num = df['num']
    stat_cls = []
    for cls, n_node in zip(classes, num):
        idx = class_2_idx(cls)
        stat_cls.append((idx, n_node))
        
    stat_cls = np.array(sorted(stat_cls, key=lambda x:x[0]))
    tong = 1.*stat_cls[:,1].sum()
    freq = stat_cls[:,1]/tong
    # alpha = 1/freq
    alpha = 1.0 - freq
    return torch.from_numpy(alpha)
alpha = cacu_alpha()
print(alpha)

tensor([0.9888, 0.9505, 0.9949, 0.9913, 0.9944, 0.9973, 0.9901, 0.9851, 0.9859,
        0.9840, 0.9851, 0.9543, 0.9950, 0.9968, 0.9975, 0.9988, 0.9945, 0.9973,
        0.9916, 0.9913, 0.9809, 0.9885, 0.9901, 0.9975, 0.9942, 0.9974, 0.9735,
        0.9631, 0.9944, 0.9816, 0.9802, 0.9525, 0.9943, 0.9944, 0.9944, 0.9944,
        0.9972, 0.9836, 0.9972, 0.9801, 0.9854, 0.9876, 0.9934, 0.9727, 0.9777,
        0.9861, 0.9941, 0.6093], dtype=torch.float64)


In [6]:
@torch.no_grad()
def _val(val_dataset:DGLDataset, model:nn.Module) -> float:
    model.eval()
    acc = 0.0
    for g in val_dataset:
        features = g.ndata['feat'].float()
        labels = g.ndata['label']
        logits = model(g, features)
        pred = logits.argmax(1)
        acc += (pred == labels).float().mean()
    return acc/len(val_dataset)

@torch.no_grad()
def val(val_dataset:DGLDataset, model:nn.Module, 
        ignore_class:Optional[Tuple[int, ...]]=None) -> float:
    if ignore_class is None: 
        return _val(val_dataset, model)
    
    model.eval()
    acc = 0.0
    for g in val_dataset:
        features = g.ndata['feat'].float()
        labels = g.ndata['label']
        logits = model(g, features)
        pred = logits.argmax(1)
        
        filter_cls = [y for y, lbl in zip(pred == labels, labels) 
                      if lbl not in ignore_class]
        acc += torch.tensor(filter_cls).float().mean()
    return acc / len(val_dataset)

@torch.enable_grad()
def train(train_dataset:DGLDataset, model:nn.Module,
          criteron:nn.Module, optimizer:optim.Optimizer) -> Tuple[nn.Module, float]:
    raise NotImplementedError('chưa test func này')
    model.train()
    acc = 0.
    for g in train_dataset:
        features = g.ndata['feat'].float()
        labels = g.ndata['label']
        logits = model(g, features)
        pred = logits.argmax(1)
        loss = criteron(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc += (pred == labels).float().mean()
    
    train_dataset.shuffle()
    train_acc = (1.*acc)/len(train_dataset)
    
    return model, train_acc

def train_n_valid(train_dataset:DGLDataset, val_dataset:DGLDataset, 
                  model:nn.Module, epochs:int, lr=0.001, path_save:str=...):
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-8)
    # optimizer = AdaBelief(model.parameters(), lr=lr, betas=(0.9,0.999), eps=1e-8, 
                        #   rectify = False, print_change_log=False)
    criteron:FocalLoss = focal_loss(alpha= alpha, gamma=4.2)
    best_acc = -1.0

    for epoch in range(1, epochs+1):
        model.train()
        acc = 0.
        for g in train_dataset:
            features = g.ndata['feat'].float()
            labels = g.ndata['label']
            
            logits = model(g, features)
            pred = logits.argmax(1)
            
            loss = criteron(logits, labels)
            acc += (pred == labels).float().mean()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        train_dataset.shuffle()
        
        train_acc = (1.*acc)/len(train_dataset)
        # val_acc = val(val_dataset, model)
        val_acc = val(val_dataset, model, None)
        if val_acc >= best_acc:
            best_acc = val_acc
            best_model = deepcopy(model)
            torch.save(model.state_dict(), path_save)
        

        print('Epoch {:<4d}: loss: {:.4f}, best {:.4f}, train_acc: {:.4f}, val_acc: {:.4f}'.
              format(epoch, loss, best_acc, train_acc, val_acc))

    torch.save(best_model.state_dict(), path_save)
    print('Last epoch {:<4d}: loss: {:.4f}, best {:.4f}, train_acc: {:.4f}, val_acc: {:.4f}'.
          format(epoch, loss, best_acc, train_acc, val_acc))
    
    return best_model, best_acc

In [7]:
num_classes = len(classes)  #48
print('num classes = ', num_classes)

num classes =  48


In [10]:
from dgl.nn.pytorch import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self,
                 in_feats,
                 n_classes,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, 352, aggregator_type, dropout, activation=activation)
        self.conv2 = SAGEConv(352, 96, aggregator_type, dropout, activation=activation)   # 196, 64 # 384, 128
        self.conv3 = SAGEConv(96, n_classes, aggregator_type, dropout, activation=activation)
        self.activation = activation

    def forward(self, graph, inputs):
        h = self.conv1(graph, inputs)
        h = self.conv2(graph, h)
        h = self.conv3(graph, h)
        return h

In [11]:
model = GraphSAGE(772, 48, F.leaky_relu, 0.4, 'mean')
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth', map_location='cpu'))
print(model)
best_model, best_acc = train_n_valid(train_data, val_data, model, 1000, lr=0.001, path_save='weights/graphSAGE_best40.pth')


GraphSAGE(
  (conv1): SAGEConv(
    (feat_drop): Dropout(p=0.4, inplace=False)
    (fc_self): Linear(in_features=772, out_features=352, bias=False)
    (fc_neigh): Linear(in_features=772, out_features=352, bias=False)
  )
  (conv2): SAGEConv(
    (feat_drop): Dropout(p=0.4, inplace=False)
    (fc_self): Linear(in_features=352, out_features=96, bias=False)
    (fc_neigh): Linear(in_features=352, out_features=96, bias=False)
  )
  (conv3): SAGEConv(
    (feat_drop): Dropout(p=0.4, inplace=False)
    (fc_self): Linear(in_features=96, out_features=48, bias=False)
    (fc_neigh): Linear(in_features=96, out_features=48, bias=False)
  )
)
Epoch 1   : loss: 3.6805, best 0.0236, train_acc: 0.0460, val_acc: 0.0236
Epoch 2   : loss: 2.9591, best 0.0673, train_acc: 0.0943, val_acc: 0.0673
Epoch 3   : loss: 3.5146, best 0.0776, train_acc: 0.0963, val_acc: 0.0776
Epoch 4   : loss: 3.0426, best 0.0889, train_acc: 0.1227, val_acc: 0.0889
Epoch 5   : loss: 4.0135, best 0.1192, train_acc: 0.1464, val_ac

In [12]:
model = GraphSAGE(772, 48, F.leaky_relu, 0.4, 'lstm')
best_model, best_acc = train_n_valid(train_data, val_data, model, 2500, lr=0.001, path_save='weights/graphSAGE_bestLSTM.pth')


Epoch 1   : loss: 3.7226, best 0.0566, train_acc: 0.0502, val_acc: 0.0566
Epoch 2   : loss: 3.5523, best 0.1609, train_acc: 0.0831, val_acc: 0.1609
Epoch 3   : loss: 3.3946, best 0.1609, train_acc: 0.1025, val_acc: 0.1282
Epoch 4   : loss: 2.7950, best 0.1609, train_acc: 0.1229, val_acc: 0.1261
Epoch 5   : loss: 2.6450, best 0.1823, train_acc: 0.1280, val_acc: 0.1823
Epoch 6   : loss: 3.3528, best 0.1823, train_acc: 0.1493, val_acc: 0.1106
Epoch 7   : loss: 3.3571, best 0.1823, train_acc: 0.1634, val_acc: 0.1715
Epoch 8   : loss: 3.5032, best 0.2033, train_acc: 0.1918, val_acc: 0.2033
Epoch 9   : loss: 2.9380, best 0.2586, train_acc: 0.2341, val_acc: 0.2586
Epoch 10  : loss: 2.6433, best 0.2883, train_acc: 0.2657, val_acc: 0.2883
Epoch 11  : loss: 0.6129, best 0.2883, train_acc: 0.2858, val_acc: 0.2388
Epoch 12  : loss: 1.0532, best 0.2883, train_acc: 0.3052, val_acc: 0.2855
Epoch 13  : loss: 1.9406, best 0.2883, train_acc: 0.3347, val_acc: 0.2452
Epoch 14  : loss: 1.9763, best 0.3134,

In [ ]:
from dgl.nn.pytorch import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self,
                 in_feats,
                 n_classes,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, 384, aggregator_type, dropout, activation=activation)
        self.conv2 = SAGEConv(384, 128, aggregator_type, dropout, activation=activation)   # 196, 64 # 384, 128
        self.conv3 = SAGEConv(128, n_classes, aggregator_type, dropout, activation=activation)
        self.activation = activation

    def forward(self, graph, inputs):
        h = self.conv1(graph, inputs)
        h = self.conv2(graph, h)
        h = self.conv3(graph, h)
        return h

In [ ]:
model = GraphSAGE(772, 48, F.leaky_relu, 0.4, 'mean')
best_model, best_acc = train_n_valid(train_data, val_data, model, 2500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


In [6]:
drop = 0.05
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 2500, lr=0.0009, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.5689, best 0.0659, train_acc: 0.0620, val_acc: 0.0659
Epoch 2   : loss: 2.4151, best 0.0659, train_acc: 0.0711, val_acc: 0.0659
Epoch 3   : loss: 1.8272, best 0.0659, train_acc: 0.0755, val_acc: 0.0645
Epoch 4   : loss: 3.2231, best 0.1371, train_acc: 0.1201, val_acc: 0.1371
Epoch 5   : loss: 2.1614, best 0.1371, train_acc: 0.0965, val_acc: 0.1140
Epoch 6   : loss: 2.6407, best 0.1721, train_acc: 0.1367, val_acc: 0.1721
Epoch 7   : loss: 2.6142, best 0.1721, train_acc: 0.1382, val_acc: 0.1160
Epoch 8   : loss: 3.1520, best 0.1721, train_acc: 0.1583, val_acc: 0.1278
Epoch 9   : loss: 1.3804, best 0.1721, train_acc: 0.1865, val_acc: 0.1494
Epoch 10  : loss: 1.9477, best 0.2262, train_acc: 0.2224, val_acc: 0.2262
Epoch 11  : loss: 2.3537, best 0.2262, train_acc: 0.2198, val_acc: 0.1658
Epoch 12  : loss: 2.2571, best 0.2349, train_acc: 0.2430, val_acc: 0.2349
Epoch 13  : loss: 1.7759, best 0.2459, train_acc: 0.2834, val_acc: 0.2459
Epoch 14  : loss: 2.2798, best 0.2459,

In [7]:
drop = 0.1
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.4040, best 0.0659, train_acc: 0.0615, val_acc: 0.0659
Epoch 2   : loss: 3.4979, best 0.0677, train_acc: 0.0821, val_acc: 0.0677
Epoch 3   : loss: 2.3593, best 0.0696, train_acc: 0.1018, val_acc: 0.0696
Epoch 4   : loss: 3.1216, best 0.0800, train_acc: 0.1083, val_acc: 0.0800
Epoch 5   : loss: 2.9582, best 0.0870, train_acc: 0.1290, val_acc: 0.0870
Epoch 6   : loss: 2.9835, best 0.0870, train_acc: 0.1221, val_acc: 0.0807
Epoch 7   : loss: 2.6082, best 0.0943, train_acc: 0.1378, val_acc: 0.0943
Epoch 8   : loss: 1.5456, best 0.1035, train_acc: 0.1608, val_acc: 0.1035
Epoch 9   : loss: 1.7440, best 0.1221, train_acc: 0.1888, val_acc: 0.1221
Epoch 10  : loss: 1.4189, best 0.1557, train_acc: 0.1942, val_acc: 0.1557
Epoch 11  : loss: 2.3278, best 0.1557, train_acc: 0.2078, val_acc: 0.1237
Epoch 12  : loss: 1.3745, best 0.1765, train_acc: 0.2344, val_acc: 0.1765
Epoch 13  : loss: 1.0132, best 0.1765, train_acc: 0.2544, val_acc: 0.1756
Epoch 14  : loss: 2.7944, best 0.1765,

In [8]:
drop = 0.2
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.7354, best 0.0208, train_acc: 0.0249, val_acc: 0.0208
Epoch 2   : loss: 3.3270, best 0.0656, train_acc: 0.0744, val_acc: 0.0656
Epoch 3   : loss: 3.5497, best 0.0775, train_acc: 0.1042, val_acc: 0.0775
Epoch 4   : loss: 3.2164, best 0.0775, train_acc: 0.1095, val_acc: 0.0351
Epoch 5   : loss: 2.7181, best 0.0811, train_acc: 0.1279, val_acc: 0.0811
Epoch 6   : loss: 2.9526, best 0.1117, train_acc: 0.1380, val_acc: 0.1117
Epoch 7   : loss: 2.8378, best 0.1117, train_acc: 0.1346, val_acc: 0.0960
Epoch 8   : loss: 1.9670, best 0.1117, train_acc: 0.1529, val_acc: 0.1028
Epoch 9   : loss: 2.3480, best 0.1433, train_acc: 0.1845, val_acc: 0.1433
Epoch 10  : loss: 2.2660, best 0.1556, train_acc: 0.2215, val_acc: 0.1556
Epoch 11  : loss: 2.6274, best 0.1710, train_acc: 0.2401, val_acc: 0.1710
Epoch 12  : loss: 2.1898, best 0.1837, train_acc: 0.2542, val_acc: 0.1837
Epoch 13  : loss: 2.2819, best 0.1881, train_acc: 0.2705, val_acc: 0.1881
Epoch 14  : loss: 2.3928, best 0.1908,

In [9]:
drop = 0.3
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.1356, best 0.0659, train_acc: 0.0548, val_acc: 0.0659
Epoch 2   : loss: 2.3726, best 0.0787, train_acc: 0.0785, val_acc: 0.0787
Epoch 3   : loss: 3.2317, best 0.0789, train_acc: 0.0884, val_acc: 0.0789
Epoch 4   : loss: 2.3536, best 0.0789, train_acc: 0.1211, val_acc: 0.0734
Epoch 5   : loss: 1.9327, best 0.0854, train_acc: 0.1079, val_acc: 0.0854
Epoch 6   : loss: 2.9991, best 0.1132, train_acc: 0.1061, val_acc: 0.1132
Epoch 7   : loss: 2.9717, best 0.1132, train_acc: 0.1532, val_acc: 0.1065
Epoch 8   : loss: 1.7003, best 0.1632, train_acc: 0.1633, val_acc: 0.1632
Epoch 9   : loss: 2.6543, best 0.1774, train_acc: 0.1799, val_acc: 0.1774
Epoch 10  : loss: 1.2316, best 0.1774, train_acc: 0.2101, val_acc: 0.1603
Epoch 11  : loss: 2.4267, best 0.1934, train_acc: 0.2281, val_acc: 0.1934
Epoch 12  : loss: 1.0900, best 0.2165, train_acc: 0.2450, val_acc: 0.2165
Epoch 13  : loss: 1.9886, best 0.2581, train_acc: 0.2593, val_acc: 0.2581
Epoch 14  : loss: 1.0893, best 0.2581,

In [10]:
drop = 0.4
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.5363, best 0.0652, train_acc: 0.0545, val_acc: 0.0652
Epoch 2   : loss: 3.4260, best 0.0652, train_acc: 0.0732, val_acc: 0.0573
Epoch 3   : loss: 3.3762, best 0.0721, train_acc: 0.0934, val_acc: 0.0721
Epoch 4   : loss: 2.9018, best 0.0891, train_acc: 0.0942, val_acc: 0.0891
Epoch 5   : loss: 2.7398, best 0.0934, train_acc: 0.1245, val_acc: 0.0934
Epoch 6   : loss: 2.6116, best 0.1166, train_acc: 0.1398, val_acc: 0.1166
Epoch 7   : loss: 2.5097, best 0.1166, train_acc: 0.1369, val_acc: 0.0754
Epoch 8   : loss: 3.0071, best 0.1166, train_acc: 0.1557, val_acc: 0.1129
Epoch 9   : loss: 2.7769, best 0.1344, train_acc: 0.1637, val_acc: 0.1344
Epoch 10  : loss: 1.4177, best 0.1775, train_acc: 0.1994, val_acc: 0.1775
Epoch 11  : loss: 2.2439, best 0.1775, train_acc: 0.2263, val_acc: 0.1593
Epoch 12  : loss: 2.3807, best 0.1795, train_acc: 0.2417, val_acc: 0.1795
Epoch 13  : loss: 2.2556, best 0.1839, train_acc: 0.2530, val_acc: 0.1839
Epoch 14  : loss: 2.3643, best 0.2320,

In [11]:
drop = 0.01
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 2.5963, best 0.0659, train_acc: 0.0563, val_acc: 0.0659
Epoch 2   : loss: 2.7024, best 0.0713, train_acc: 0.0687, val_acc: 0.0713
Epoch 3   : loss: 2.4424, best 0.0713, train_acc: 0.0850, val_acc: 0.0570
Epoch 4   : loss: 2.2067, best 0.0713, train_acc: 0.0864, val_acc: 0.0641
Epoch 5   : loss: 3.7267, best 0.0713, train_acc: 0.1256, val_acc: 0.0612
Epoch 6   : loss: 2.8288, best 0.1644, train_acc: 0.1434, val_acc: 0.1644
Epoch 7   : loss: 2.3658, best 0.1644, train_acc: 0.1672, val_acc: 0.0961
Epoch 8   : loss: 2.7946, best 0.1646, train_acc: 0.1846, val_acc: 0.1646
Epoch 9   : loss: 2.8000, best 0.1646, train_acc: 0.1785, val_acc: 0.1644
Epoch 10  : loss: 1.2601, best 0.2190, train_acc: 0.2283, val_acc: 0.2190
Epoch 11  : loss: 1.9918, best 0.2190, train_acc: 0.2548, val_acc: 0.1749
Epoch 12  : loss: 2.2934, best 0.2371, train_acc: 0.2697, val_acc: 0.2371
Epoch 13  : loss: 2.3256, best 0.2371, train_acc: 0.2815, val_acc: 0.2266
Epoch 14  : loss: 2.3192, best 0.2442,

In [12]:
drop = 0.006
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 2.9506, best 0.0659, train_acc: 0.0499, val_acc: 0.0659
Epoch 2   : loss: 2.8436, best 0.0659, train_acc: 0.0658, val_acc: 0.0208
Epoch 3   : loss: 3.0301, best 0.0659, train_acc: 0.0831, val_acc: 0.0208
Epoch 4   : loss: 3.5732, best 0.0811, train_acc: 0.1199, val_acc: 0.0811
Epoch 5   : loss: 2.9187, best 0.1001, train_acc: 0.1119, val_acc: 0.1001
Epoch 6   : loss: 2.2050, best 0.1289, train_acc: 0.1288, val_acc: 0.1289
Epoch 7   : loss: 2.7553, best 0.1289, train_acc: 0.1424, val_acc: 0.0997
Epoch 8   : loss: 2.4836, best 0.1289, train_acc: 0.1853, val_acc: 0.1061
Epoch 9   : loss: 2.3662, best 0.1518, train_acc: 0.1815, val_acc: 0.1518
Epoch 10  : loss: 2.3507, best 0.1518, train_acc: 0.2234, val_acc: 0.1423
Epoch 11  : loss: 1.2489, best 0.1582, train_acc: 0.2490, val_acc: 0.1582
Epoch 12  : loss: 2.5156, best 0.1582, train_acc: 0.2715, val_acc: 0.1423
Epoch 13  : loss: 3.3934, best 0.1822, train_acc: 0.2808, val_acc: 0.1822
Epoch 14  : loss: 1.9592, best 0.1851,

In [13]:
drop = 0.004
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.0745, best 0.0659, train_acc: 0.0575, val_acc: 0.0659
Epoch 2   : loss: 3.3704, best 0.0785, train_acc: 0.0692, val_acc: 0.0785
Epoch 3   : loss: 2.8546, best 0.0785, train_acc: 0.1105, val_acc: 0.0651
Epoch 4   : loss: 3.0814, best 0.0788, train_acc: 0.1067, val_acc: 0.0788
Epoch 5   : loss: 2.2668, best 0.0788, train_acc: 0.1058, val_acc: 0.0761
Epoch 6   : loss: 1.2106, best 0.1248, train_acc: 0.1508, val_acc: 0.1248
Epoch 7   : loss: 1.9454, best 0.1577, train_acc: 0.1426, val_acc: 0.1577
Epoch 8   : loss: 3.3515, best 0.1577, train_acc: 0.2047, val_acc: 0.1519
Epoch 9   : loss: 2.3099, best 0.1827, train_acc: 0.2010, val_acc: 0.1827
Epoch 10  : loss: 3.4633, best 0.1827, train_acc: 0.1920, val_acc: 0.1393
Epoch 11  : loss: 2.3218, best 0.2146, train_acc: 0.2713, val_acc: 0.2146
Epoch 12  : loss: 1.9423, best 0.2146, train_acc: 0.2589, val_acc: 0.1599
Epoch 13  : loss: 1.8719, best 0.2146, train_acc: 0.2949, val_acc: 0.1796
Epoch 14  : loss: 3.2553, best 0.2146,

In [6]:
drop = 0.35
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.2423, best 0.0904, train_acc: 0.0445, val_acc: 0.0904
Epoch 2   : loss: 2.2274, best 0.0904, train_acc: 0.0684, val_acc: 0.0770
Epoch 3   : loss: 3.5517, best 0.0904, train_acc: 0.0843, val_acc: 0.0501
Epoch 4   : loss: 3.2314, best 0.0935, train_acc: 0.1207, val_acc: 0.0935
Epoch 5   : loss: 3.1660, best 0.0935, train_acc: 0.1146, val_acc: 0.0815
Epoch 6   : loss: 1.9369, best 0.0935, train_acc: 0.1254, val_acc: 0.0847
Epoch 7   : loss: 2.5679, best 0.1042, train_acc: 0.1476, val_acc: 0.1042
Epoch 8   : loss: 1.6224, best 0.1450, train_acc: 0.1734, val_acc: 0.1450
Epoch 9   : loss: 2.7904, best 0.1450, train_acc: 0.1903, val_acc: 0.1283
Epoch 10  : loss: 2.5079, best 0.1532, train_acc: 0.2117, val_acc: 0.1532
Epoch 11  : loss: 2.2404, best 0.1734, train_acc: 0.2293, val_acc: 0.1734
Epoch 12  : loss: 2.3806, best 0.2416, train_acc: 0.2542, val_acc: 0.2416
Epoch 13  : loss: 0.8162, best 0.2416, train_acc: 0.2572, val_acc: 0.2358
Epoch 14  : loss: 2.2196, best 0.2416,

In [7]:
drop = 0.45
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 2.9916, best 0.0659, train_acc: 0.0455, val_acc: 0.0659
Epoch 2   : loss: 2.9344, best 0.0659, train_acc: 0.0699, val_acc: 0.0332
Epoch 3   : loss: 2.3269, best 0.0797, train_acc: 0.0818, val_acc: 0.0797
Epoch 4   : loss: 2.7018, best 0.0814, train_acc: 0.0896, val_acc: 0.0814
Epoch 5   : loss: 3.4005, best 0.0814, train_acc: 0.0939, val_acc: 0.0591
Epoch 6   : loss: 2.6066, best 0.1506, train_acc: 0.1247, val_acc: 0.1506
Epoch 7   : loss: 2.2336, best 0.1506, train_acc: 0.1226, val_acc: 0.0884
Epoch 8   : loss: 1.7311, best 0.1506, train_acc: 0.1474, val_acc: 0.1261
Epoch 9   : loss: 2.3541, best 0.1879, train_acc: 0.1739, val_acc: 0.1879
Epoch 10  : loss: 2.1799, best 0.1879, train_acc: 0.1959, val_acc: 0.1693
Epoch 11  : loss: 2.5291, best 0.1879, train_acc: 0.2081, val_acc: 0.1775
Epoch 12  : loss: 2.1113, best 0.2050, train_acc: 0.2196, val_acc: 0.2050
Epoch 13  : loss: 0.8416, best 0.2050, train_acc: 0.2377, val_acc: 0.1750
Epoch 14  : loss: 1.7683, best 0.2603,

In [8]:
drop = 0.5
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.6447, best 0.0659, train_acc: 0.0478, val_acc: 0.0659
Epoch 2   : loss: 1.7996, best 0.0665, train_acc: 0.0848, val_acc: 0.0665
Epoch 3   : loss: 3.3222, best 0.0796, train_acc: 0.1056, val_acc: 0.0796
Epoch 4   : loss: 2.6292, best 0.0840, train_acc: 0.1159, val_acc: 0.0840
Epoch 5   : loss: 2.7008, best 0.0840, train_acc: 0.1317, val_acc: 0.0815
Epoch 6   : loss: 2.2201, best 0.0875, train_acc: 0.1242, val_acc: 0.0875
Epoch 7   : loss: 1.8764, best 0.0887, train_acc: 0.1402, val_acc: 0.0887
Epoch 8   : loss: 3.4394, best 0.1186, train_acc: 0.1493, val_acc: 0.1186
Epoch 9   : loss: 2.9299, best 0.1186, train_acc: 0.1567, val_acc: 0.1130
Epoch 10  : loss: 2.9346, best 0.1852, train_acc: 0.1785, val_acc: 0.1852
Epoch 11  : loss: 2.2878, best 0.1852, train_acc: 0.1996, val_acc: 0.1352
Epoch 12  : loss: 0.4381, best 0.1852, train_acc: 0.2081, val_acc: 0.1657
Epoch 13  : loss: 2.1020, best 0.2114, train_acc: 0.2355, val_acc: 0.2114
Epoch 14  : loss: 1.1920, best 0.2114,

In [9]:
drop = 0.55
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.6183, best 0.0659, train_acc: 0.0582, val_acc: 0.0659
Epoch 2   : loss: 3.5431, best 0.0773, train_acc: 0.0674, val_acc: 0.0773
Epoch 3   : loss: 3.2589, best 0.0773, train_acc: 0.0689, val_acc: 0.0350
Epoch 4   : loss: 3.0727, best 0.0773, train_acc: 0.0941, val_acc: 0.0724
Epoch 5   : loss: 2.4398, best 0.0789, train_acc: 0.1210, val_acc: 0.0789
Epoch 6   : loss: 3.2104, best 0.0888, train_acc: 0.0925, val_acc: 0.0888
Epoch 7   : loss: 2.1906, best 0.1105, train_acc: 0.1059, val_acc: 0.1105
Epoch 8   : loss: 2.6389, best 0.1277, train_acc: 0.1260, val_acc: 0.1277
Epoch 9   : loss: 1.8704, best 0.1277, train_acc: 0.1531, val_acc: 0.1070
Epoch 10  : loss: 2.8057, best 0.1305, train_acc: 0.1636, val_acc: 0.1305
Epoch 11  : loss: 3.1244, best 0.1400, train_acc: 0.1682, val_acc: 0.1400
Epoch 12  : loss: 2.9269, best 0.1543, train_acc: 0.1999, val_acc: 0.1543
Epoch 13  : loss: 2.8919, best 0.1543, train_acc: 0.1979, val_acc: 0.1345
Epoch 14  : loss: 2.4344, best 0.2264,

In [10]:
drop = 0.6
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 3.5656, best 0.0659, train_acc: 0.0525, val_acc: 0.0659
Epoch 2   : loss: 3.0507, best 0.0659, train_acc: 0.0816, val_acc: 0.0600
Epoch 3   : loss: 3.6384, best 0.0759, train_acc: 0.0800, val_acc: 0.0759
Epoch 4   : loss: 3.1876, best 0.0811, train_acc: 0.0953, val_acc: 0.0811
Epoch 5   : loss: 3.1831, best 0.0933, train_acc: 0.1143, val_acc: 0.0933
Epoch 6   : loss: 3.5295, best 0.0933, train_acc: 0.1339, val_acc: 0.0796
Epoch 7   : loss: 2.9480, best 0.1034, train_acc: 0.1160, val_acc: 0.1034
Epoch 8   : loss: 2.0963, best 0.1170, train_acc: 0.1474, val_acc: 0.1170
Epoch 9   : loss: 2.7855, best 0.1222, train_acc: 0.1656, val_acc: 0.1222
Epoch 10  : loss: 2.8229, best 0.1299, train_acc: 0.1706, val_acc: 0.1299
Epoch 11  : loss: 3.3830, best 0.1610, train_acc: 0.1814, val_acc: 0.1610
Epoch 12  : loss: 2.7918, best 0.1610, train_acc: 0.2014, val_acc: 0.1580
Epoch 13  : loss: 1.3388, best 0.2298, train_acc: 0.2336, val_acc: 0.2298
Epoch 14  : loss: 2.3155, best 0.2325,

In [11]:
drop = 0.65
model = GraphSAGE(772, 196, 64, drop, num_classes) #
# model.load_state_dict(torch.load('weights/graphSAGE_best40.pth',
#                                  map_location=torch.device('cpu')))
best_model, best_acc = train_n_valid(train_data, val_data, model, 1500, lr=0.001, path_save='weights/graphSAGE_best40.pth')


Epoch 1   : loss: 2.8783, best 0.0659, train_acc: 0.0547, val_acc: 0.0659
Epoch 2   : loss: 3.0119, best 0.0738, train_acc: 0.0669, val_acc: 0.0738
Epoch 3   : loss: 3.4876, best 0.0784, train_acc: 0.0785, val_acc: 0.0784
Epoch 4   : loss: 2.3399, best 0.0830, train_acc: 0.1028, val_acc: 0.0830
Epoch 5   : loss: 3.1446, best 0.0830, train_acc: 0.1098, val_acc: 0.0803
Epoch 6   : loss: 2.1931, best 0.0938, train_acc: 0.0930, val_acc: 0.0938
Epoch 7   : loss: 2.6223, best 0.0938, train_acc: 0.1266, val_acc: 0.0856
Epoch 8   : loss: 1.5437, best 0.1138, train_acc: 0.1284, val_acc: 0.1138
Epoch 9   : loss: 3.5901, best 0.1138, train_acc: 0.1307, val_acc: 0.1058
Epoch 10  : loss: 2.3774, best 0.1387, train_acc: 0.1610, val_acc: 0.1387
Epoch 11  : loss: 2.6483, best 0.1788, train_acc: 0.1746, val_acc: 0.1788
Epoch 12  : loss: 3.5166, best 0.1788, train_acc: 0.1911, val_acc: 0.1733
Epoch 13  : loss: 2.5928, best 0.1788, train_acc: 0.2062, val_acc: 0.1527
Epoch 14  : loss: 2.6930, best 0.1788,